In [6]:
# 载入所需的包
import sqlite3
import pymongo

In [7]:
# 数据库初始化连接

# 垃圾sqlite初始化
sqlite_db = sqlite3.connect("./fe/data/book_lx.db")
cur_s = sqlite_db.cursor()

# 芒果初始化
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client['bookstore']
cur_m = db['book']

In [8]:
def data_construct(line):  
    one_data = {  
        "id": line[0],  
        "title": line[1],  
        "author": line[2],  
        "publisher": line[3],  
        "original_title": line[4],  
        "translator": line[5],  
        "pub_year": line[6],  
        "pages": line[7],  
        "price": line[8],  
        "currency_unit": line[9],  
        "binding": line[10],  
        "isbn": line[11],  
        "author_intro": line[12],  
        "book_intro": line[13],  
        "content": line[14],  
        "tags": line[15],
        "picture": line[16]
    }  
    return one_data

In [9]:
sql = "select id, title, author, publisher, original_title, translator, pub_year, pages, price, currency_unit, binding, isbn, author_intro, book_intro, content, tags, picture from book"
result = cur_s.execute(sql)
books_data = []  # 储存即将插入mongo的数据
for row in result:
    books_data.append(data_construct(row))
cur_m.insert_many(books_data)

In [14]:
content = cur_m.find()
count = 0
for each in content:
    count += 1
print(count)

43988


In [10]:
import sqlite3
import pymongo
import random

def data_construct(store_id, book_id, book_info):
    return {
        "store_id": store_id,
        "book_id": book_id,
        "book_info": book_info,
        "stock_level": random.randint(1, 1000)
    }

# 数据库初始化连接

# SQLite初始化
sqlite_db = sqlite3.connect("./fe/data/be.db")
cur_s = sqlite_db.cursor()

# MongoDB初始化
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client['bookstore']

# 从SQLite获取所有的store_id
sql = "select store_id from user_store"
store_ids = [row[0] for row in cur_s.execute(sql)]

# 从MongoDB获取所有的book_id和tags作为book_info
book_collection = db['book']
books = list(book_collection.find({}, {"id": 1, "tags": 1}))

book_data = []

# 为每个store_id赋予所有的book_id
all_books_ids = {book['id'] for book in books}
store_books = {store_id: set(all_books_ids) for store_id in store_ids}

# 随机选择一些store_id并删除其中的一些book_id
selected_store_ids = random.sample(store_ids, k=int(len(store_ids) * 0.5))
for store_id in selected_store_ids:
    num_to_remove = random.randint(1, len(all_books_ids) - 1)
    books_to_remove = random.sample(list(all_books_ids), k=num_to_remove)  # Change made here
    store_books[store_id].difference_update(books_to_remove)

for store_id, book_ids in store_books.items():
    for book_id in book_ids:
        book_info = next(b['tags'] for b in books if b['id'] == book_id)
        book_data.append(data_construct(store_id, book_id, book_info))

db['store'].insert_many(book_data)


In [11]:
# 载入所需的包
import sqlite3
import pymongo
import uuid
import random  # 引入random模块


def data_construct(order_id, user_id, store_id):  
    one_data = {  
        "order_id": order_id,
        "user_id": user_id,
        "store_id": store_id
    }  
    return one_data


# 数据库初始化连接

# 垃圾sqlite初始化
sqlite_db = sqlite3.connect("./fe/data/be.db")
cur_s = sqlite_db.cursor()

# 芒果初始化
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client['bookstore']
cur_m = db['new_order']

# 查询所有的user-store组合
sql = "select user_id, store_id from user_store"
user_store_pairs = set(cur_s.execute(sql).fetchall())

# 查询所有可能的store_id
all_stores = set([item[0] for item in cur_s.execute("select distinct store_id from user_store").fetchall()])

books_data = []  # 储存即将插入mongo的数据

# 对于每一个user_id，选择一个与其不匹配的store_id
for user_id, own_store_id in user_store_pairs:
    available_stores = all_stores - {own_store_id}  # 排除掉自己拥有的店铺
    chosen_store_id = random.choice(list(available_stores))
    
    order_id = str(uuid.uuid4())  # 为每条数据生成一个新的UUID
    books_data.append(data_construct(order_id, user_id, chosen_store_id))

cur_m.insert_many(books_data)